In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options

import time
import re

from bs4 import BeautifulSoup

In [20]:
data_base_path = "C:\Potenup\Drug-Detection-Chatbot\data\\"

In [21]:
original_file_name = "단일 경구약제_5,000종 리스트.xlsx"
file_name = "single_drug_list_5000.csv"

# 약 리스트 데이터 불러오기

In [ ]:
df_1 = pd.read_excel(data_base_path + original_file_name, sheet_name=0)

In [4]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   C-
Code       1000 non-null   object 
 1   구분            1000 non-null   object 
 2   제품명           1000 non-null   object 
 3   복지부 
분류코드     1000 non-null   float64
 4   Unnamed: 4    0 non-null      float64
 5   Unnamed: 5    0 non-null      float64
 6   Unnamed: 6    0 non-null      float64
 7   Unnamed: 7    0 non-null      float64
 8   Unnamed: 8    0 non-null      float64
 9   Conncet
Code  5000 non-null   object 
 10  구분.1          5000 non-null   object 
 11  제품명.1         5000 non-null   object 
 12  복지부
분류코드      5000 non-null   int64  
dtypes: float64(6), int64(1), object(6)
memory usage: 507.9+ KB


In [ ]:
df_2 = pd.read_excel(data_base_path + original_file_name, sheet_name=1)

In [6]:
df_2.head()

,C-\nCode,구분,제품명,복지부 \n분류코드,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Conncet\nCode,구분.1,제품명.1,복지부\n분류코드
0,K-000027,일반의약품,아네모정 20T,234.0,NaN,NaN,NaN,NaN,NaN,K-000032,전문의약품,다이크로짇정 25MG (유한),213
1,K-000030,전문의약품,테라싸이클린캅셀 250MG (종근당),615.0,NaN,NaN,NaN,NaN,NaN,K-000034,일반의약품,페니라민정,141
2,K-000041,일반의약품,삐콤정 (유한),316.0,NaN,NaN,NaN,NaN,NaN,K-000040,전문의약품,유한짓정 (유한),622
3,K-000042,일반의약품,푸라콩정,141.0,NaN,NaN,NaN,NaN,NaN,K-000043,전문의약품,디고신정 0.25MG(HK이노엔),211
4,K-000065,전문의약품,크로세린캅셀 250MG (동아),613.0,NaN,NaN,NaN,NaN,NaN,K-000117,일반의약품,액티피드정 (삼일),141


In [7]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   C-
Code       3999 non-null   object 
 1   구분            3999 non-null   object 
 2   제품명           3999 non-null   object 
 3   복지부 
분류코드     3999 non-null   float64
 4   Unnamed: 4    0 non-null      float64
 5   Unnamed: 5    0 non-null      float64
 6   Unnamed: 6    0 non-null      float64
 7   Unnamed: 7    0 non-null      float64
 8   Unnamed: 8    0 non-null      float64
 9   Conncet
Code  5000 non-null   object 
 10  구분.1          5000 non-null   object 
 11  제품명.1         5000 non-null   object 
 12  복지부
분류코드      5000 non-null   int64  
dtypes: float64(6), int64(1), object(6)
memory usage: 507.9+ KB


# 약 리스트 데이터 전처리

## 결측치 제거

In [ ]:
# 필요 없는 칼럼 제거
df_1 = df_1.drop(["Unnamed: 4", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Conncet\nCode", "구분.1", "제품명.1", "복지부\n분류코드"], axis=1)

In [9]:
df_1.head()

,C-\nCode,구분,제품명,복지부 \n분류코드
0,K-000059,일반의약품,게루삼정 200T,234.0
1,K-000069,일반의약품,지엘타이밍정 12T,115.0
2,K-000080,전문의약품,마이암부톨제피정400mg,622.0
3,K-000112,일반의약품,아로나민골드,316.0
4,K-000114,전문의약품,셉트린정 480MG(삼일),621.0


In [10]:
df_1.shape

(5000, 4)

In [11]:
df_1 = df_1.dropna()

In [12]:
df_1.shape

(1000, 4)

In [13]:
df_2 = df_2.drop(["Unnamed: 4", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Conncet\nCode", "구분.1", "제품명.1", "복지부\n분류코드"], axis=1)

In [14]:
df_2.shape

(5000, 4)

In [15]:
df_2 = df_2.dropna()

In [16]:
df_2.shape

(3999, 4)

## 병합

In [ ]:
# sheet1 과 sheet2 를 병합
original_df = pd.concat([df_1, df_2], axis=0)

In [18]:
original_df

,C-\nCode,구분,제품명,복지부 \n분류코드
0,K-000059,일반의약품,게루삼정 200T,234.0
1,K-000069,일반의약품,지엘타이밍정 12T,115.0
2,K-000080,전문의약품,마이암부톨제피정400mg,622.0
3,K-000112,일반의약품,아로나민골드,316.0
4,K-000114,전문의약품,셉트린정 480MG(삼일),621.0
...,...,...,...,...
3994,K-051831,일반의약품,위엔싹에프정 10T,232.0
3995,K-052494,일반의약품,뉴락스에프정 10T,238.0
3996,K-052505,전문의약품,휴메트린정0.125mg,252.0
3997,K-053315,전문의약품,레코미드서방정 150MG(유한),232.0


In [22]:
original_df.to_csv(data_base_path + file_name, index=False)

## 제품명 정제 (약명 / 용량)

In [24]:
df = pd.read_csv(data_base_path + file_name)

In [25]:
df.head()

,C-Code,구분,제품명,복지부분류코드
0,K-000059,일반의약품,게루삼정 200T,234.0
1,K-000069,일반의약품,지엘타이밍정 12T,115.0
2,K-000080,전문의약품,마이암부톨제피정400mg,622.0
3,K-000112,일반의약품,아로나민골드,316.0
4,K-000114,전문의약품,셉트린정 480MG(삼일),621.0


In [ ]:
# 제품명에서 약의 이름만 정제
def cleaned_name(name):
    s = re.sub(r"\([^)]*\)", "", name)
    s = re.sub(r"\s*\d+\s*[A-Za-z가-힣]*", "", s)
    s = re.sub(r"[-+*/.,:;\s]", "", s)
    s = s.strip()

    return s

In [ ]:
# 제품명에서 약의 용량만 정제
def find_volume(name):
    pattern = re.compile(r"\d+\s*[A-Za-z가-힣]+")
    match = pattern.search(name)
    if match:
        return match.group()
    else:
        return ''

In [30]:
cleaned_name_list = []
volume_list = []
for i, data in df.iterrows():
    code, _, name, _ = data
    cleaned_name_list.append(cleaned_name(name))
    volume_list.append(find_volume(name))
    print("정제 : ", cleaned_name(name), ", 용량 : ", find_volume(name))

정제 :  게루삼정 , 용량 :  200T
정제 :  지엘타이밍정 , 용량 :  12T
정제 :  마이암부톨제피정 , 용량 :  400mg
정제 :  아로나민골드 , 용량 :  
정제 :  셉트린정 , 용량 :  480MG
정제 :  자이로릭정 , 용량 :  100MG
정제 :  다오닐정 , 용량 :  5mg
정제 :  보나링에이정 , 용량 :  
정제 :  에나폰정 , 용량 :  10mg
정제 :  키모랄에스정 , 용량 :  
정제 :  후릭스정 , 용량 :  40mg
정제 :  리단정 , 용량 :  300mg
정제 :  마그밀정 , 용량 :  500mg
정제 :  낙센정 , 용량 :  250mg
정제 :  구주스피로닥톤정 , 용량 :  25mg
정제 :  페리나정 , 용량 :  25mg
정제 :  로프민캡슐 , 용량 :  10C
정제 :  세가톤트로키정 , 용량 :  12T
정제 :  후시딘정 , 용량 :  250mg
정제 :  네오메디코푸정 , 용량 :  
정제 :  피라진아미드정 , 용량 :  500MG
정제 :  노이시린에이정 , 용량 :  500T
정제 :  쎄닐톤정 , 용량 :  
정제 :  메티마졸정 , 용량 :  
정제 :  게보린 , 용량 :  10T
정제 :  싸이메트정 , 용량 :  300mg
정제 :  슈다페드정 , 용량 :  
정제 :  베라드정 , 용량 :  10T
정제 :  프리미돈정 , 용량 :  250MG
정제 :  트렌탈서방정 , 용량 :  400MG
정제 :  뮤코론캅셀 , 용량 :  
정제 :  베노라제정 , 용량 :  10T
정제 :  우루사정 , 용량 :  100MG
정제 :  베타록정 , 용량 :  100MG
정제 :  스파부틴정 , 용량 :  30T
정제 :  리포덱스정 , 용량 :  450MG
정제 :  타라부틴정 , 용량 :  100mg
정제 :  디스토시드정 , 용량 :  600MG
정제 :  리팜핀정 , 용량 :  600mg
정제 :  부루펜정 , 용량 :  400mg
정제 :  파자임 , 용량 :  95 이

In [31]:
df['name'] = cleaned_name_list
df['volume'] = volume_list

In [32]:
df.head()

,C-Code,구분,제품명,복지부분류코드,name,volume
0,K-000059,일반의약품,게루삼정 200T,234.0,게루삼정,200T
1,K-000069,일반의약품,지엘타이밍정 12T,115.0,지엘타이밍정,12T
2,K-000080,전문의약품,마이암부톨제피정400mg,622.0,마이암부톨제피정,400mg
3,K-000112,일반의약품,아로나민골드,316.0,아로나민골드,
4,K-000114,전문의약품,셉트린정 480MG(삼일),621.0,셉트린정,480MG


In [36]:
df['name'].value_counts()

name
씬지로이드정     7
씬지록신정      7
자디앙듀오정     6
아모잘탄큐정     6
트리레보정      6
          ..
파모타제정      1
위엔싹에프정     1
뉴락스에프정     1
휴메트린정      1
네오메디코푸정    1
Name: count, Length: 4292, dtype: int64

# 약 검색 HTML 크롤링

약에 대한 정보를 검색 후 HTML 

## 크롬 웹 드라이버 생성

In [ ]:
def make_driver():
    opts = Options()
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("accept-language=ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7")
    opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/124.0.0.0 Safari/537.36")
    driver = uc.Chrome(options=opts)
    return driver

In [ ]:
main_url = "https://www.health.kr/main.asp"

In [ ]:
# 드라이버 생성
driver = make_driver()

time.sleep(2)

driver.get(main_url)

NameError: name 'make_driver' is not defined

## 검색 박스

In [ ]:
keyword = ""

In [ ]:
input_box_css = "#search_word"
input_box = driver.find_element(By.CSS_SELECTOR, input_box_css)
input_box.clear()
input_box.send_keys(keyword)
input_box.send_keys(Keys.ENTER)

In [ ]:
html = driver.page_source

# 약 리스트 HTML 정제

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
tds_with_onclick = [
    td for td in soup.find_all("td", class_="txtL") if td.has_attr("onclick")
]

pattern = re.compile(r"\('([^']+)'\)")  # 작은따옴표 안의 문자열만 추출
for td in tds_with_onclick:
    onclick_val = td["onclick"]
    match = pattern.search(onclick_val)
    if match:
        param = match.group(1)
        print(td.text, param)

엔비젯정10/10mg 2021021900249
엔비젯정10/20mg 2021021900250
엔비젯정10/40mg 2021021900251


# 약 세부정보 크롤링

In [ ]:
result_drug_base_url = "https://www.health.kr/searchDrug/result_drug.asp?drug_cd="
result_take_base_url = "https://www.health.kr/searchDrug/result_take.asp?drug_cd="
result_sunb_base_url = "https://www.health.kr/searchDrug/result_sunb.asp?drug_cd="
result_interaction_base_url = "https://www.health.kr/searchDrug/result_interaction.asp?drug_cd="
id = "2021021900249"